## Non-maximum Supression (NMS)

In [36]:
import torch

In [73]:
def soft_nms_implementation(dbox_data):
    x1_initial = dbox_data[:, 0]
    y1_initial = dbox_data[:, 1]
    x2_initial = dbox_data[:, 2]
    y2_initial = dbox_data[:, 3]
    
    area = (x2_initial - x1_initial + 1) * (y2_initial - y1_initial + 1)
    dbox_data = torch.cat((dbox_data, area[:, None]), dim = 1)
    
    final_dbox = []
    
    # Get position of detection box with maximum confidence score
    max_index = torch.argmax(dbox_data[:, 4], axis = 0)
    
    dbox_data[[0, max_index], :] = dbox_data[[max_index, 0], :]
    final_dbox.append(dbox_data[0, :-1])
    
    xx1 = torch.max(dbox_data[0, 0], dbox_data[1:, 0])
    yy1 = torch.max(dbox_data[0, 1], dbox_data[1:, 1])
    xx2 = torch.min(dbox_data[0, 2], dbox_data[1:, 2])
    yy2 = torch.min(dbox_data[0, 3], dbox_data[1:, 3])
    
    x_diff = xx2 - xx1 + 1
    y_diff = yy2 - yy1 + 1
      
    width = torch.max(x_diff, torch.tensor(0.0, device = "cpu"))
    height = torch.max(y_diff, torch.tensor(0.0, device = "cpu"))
    
    intersection_area = width * height
    iou = intersection_area / (dbox_data[0, 5] + dbox_data[1:, 5] - intersection_area)

In [74]:
# (x1, y1, x2, y2, confidence score)
first_detection_boxes_data = torch.tensor([[2, 2, 5, 6, 0.8], [3, 1, 5, 5, 0.1], [4, 4, 6, 7, 0.9]],
                                    dtype = torch.float)
second_detection_boxes_data = torch.tensor([[100., 100., 500., 400., 0.85], [450., 350., 700., 600., 0.45],
                                       [600., 100., 800., 300., 0.2]],
                                      dtype = torch.float)

first_detection_boxes_data = first_detection_boxes_data.to("cpu")
second_detection_boxes_data = second_detection_boxes_data.to("cpu")

In [75]:
soft_nms_implementation(first_detection_boxes_data)

tensor([[ 4.0000,  4.0000,  6.0000,  7.0000,  0.9000, 12.0000],
        [ 3.0000,  1.0000,  5.0000,  5.0000,  0.1000, 15.0000],
        [ 2.0000,  2.0000,  5.0000,  6.0000,  0.8000, 20.0000]])
tensor(12.) tensor([15., 20.])
